In [1]:
import numpy 
numpy.random.seed(1337)

In [4]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras_contrib.layers.advanced_activations import PELU, SReLU

In [6]:
batch_size=128 
num_classes=4
epochs=5
class_weights={1:350, 2:30, 3: 10}

In [7]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [8]:
os.chdir("D:");
path="D:/Dataset_Final(March)/Level_0";
classes=os.listdir(path)
x=[]#Datapoints 
y=[]#labels 
for fol in classes:
    imgfiles=os.listdir(path+u'\\'+fol);
    for img in imgfiles:
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        x.append(im)
        y.append(fol)
x=numpy.array(x)
y=numpy.array(y)

In [9]:
print(x.shape)
print(y.shape)
x=x.reshape((-1,100,100,1))

(66704, 100, 100)
(66704,)


In [10]:
n=x.shape[0]
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

In [11]:
test_split=round(n*2/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(44469, 100, 100, 1)
(44469,)
(22235, 100, 100, 1)
(22235,)


In [12]:
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

array([[ 0.,  0.,  0.,  1.]])

In [13]:
# inputs=Input(shape=(100,100,1))
# x=Conv2D(128, (3, 3), padding='same')(inputs)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(64, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(12, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)#---------------------------------------------->0.38(Epoch_70)
# x=Dense(num_classes)(x)
# x=SReLU()(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)

# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [14]:
inputs=Input(shape=(100,100,1))
x=Conv2D(32, (3, 3), padding='same')(inputs)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Conv2D(16, (3, 3), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(32, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
x=Dropout(0.2)(x)#---------------------------------------------->0.38(Epoch_70)
x=Flatten()(x)
x=Dense(num_classes)(x)

output=Activation('softmax')(x)
model=Model([inputs], output)

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
s_re_lu_1 (SReLU)            (None, 100, 100, 32)      1280000   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 16)        4624      
_________________________________________________________________
s_re_lu_2 (SReLU)            (None, 50, 50, 16)        160000    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 16)        0         
__________

In [16]:
opt=keras.optimizers.Adam(lr=1e-3)

In [17]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [18]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')

In [19]:
hist=model.fit(x_train, y_train, 
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint],
               class_weight=class_weights
               )

Train on 44469 samples, validate on 22235 samples
Epoch 1/5
44469/44469 [==============================] - 1808s 41ms/step - loss: 2.9182 - acc: 0.9524 - val_loss: 0.0832 - val_acc: 0.9713

Epoch 00001: saving model to D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/weights-improvement- 1-0.97.hdf5
Epoch 2/5
44469/44469 [==============================] - 1794s 40ms/step - loss: 1.6100 - acc: 0.9667 - val_loss: 0.0568 - val_acc: 0.9817

Epoch 00002: saving model to D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/weights-improvement- 2-0.98.hdf5
Epoch 3/5
44469/44469 [==============================] - 1794s 40ms/step - loss: 1.0290 - acc: 0.9785 - val_loss: 0.0377 - val_acc: 0.9877

Epoch 00003: saving model to D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/weights-improvement- 3-0.99.hdf5
Epoch 4/5
44469/44469 [==============================] - 1796s 40ms/step - loss: 0.7139 - acc: 0.9848 - val_loss: 0.0282 - val_acc: 0.9922

Epoch 00004: saving model to D:Dataset_Final(Mar

In [26]:
score=model.evaluate(x_test, y_test)

22235/22235 [==============================] - 219s 10ms/step


In [31]:
score[1]

0.99316393076663123

In [32]:
# y_pred=model.predict(x_test)# for dataset of 12 characters
# print (y_pred)
# y_pred=numpy.argmax(y_pred, axis=1)
# print (y_pred)

In [33]:
# model.load_weights("D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/weights-improvement- 1-0.98.hdf5")

In [34]:
# opt=keras.optimizers.Adam(lr=1e-3)

In [35]:
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [36]:
# score=model.evaluate(x_test, y_test)

In [37]:
# score[1]

In [38]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  3.18220188e-08   1.07281584e-09   9.99824464e-01   1.75511959e-04]
 [  2.15332797e-13   5.66837922e-19   3.33338446e-13   1.00000000e+00]
 [  2.71200786e-13   7.12810889e-16   1.63863593e-11   1.00000000e+00]
 ..., 
 [  6.11118144e-14   2.95622969e-17   4.08637880e-12   1.00000000e+00]
 [  6.06166395e-10   9.80048121e-15   3.94004514e-04   9.99606073e-01]
 [  2.61290865e-07   7.47299040e-08   1.48815557e-01   8.51184070e-01]]
[2 3 3 ..., 3 3 3]


In [39]:
print(confusion_matrix(numpy.argmax(y_test, axis=1),y_pred))

[[   51     3     0]
 [    6   919    34]
 [    0   109 21113]]


In [40]:

print (classification_report(numpy.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

          1       0.89      0.94      0.92        54
          2       0.89      0.96      0.92       959
          3       1.00      0.99      1.00     21222

avg / total       0.99      0.99      0.99     22235



In [21]:
import pandas

pandas.DataFrame(hist.history).to_csv("D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/Figure/History-99.csv")        
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

In [23]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

pp=PdfPages("D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/Figure/Loss-99.pdf")
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available)
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']


In [24]:
pp=PdfPages("D:Dataset_Final(March)/Checkpoints/Level_0/Epoch_22/Figure/Accuracy.pdf")
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()